# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
# Import API key
from api_key import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

613

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [11]:
# Building weather_url
url ="http://api.openweathermap.org/data/2.5/weather?"
units="metrics"

# base_url = "http://api.openweathermap.org/data/2.5/weather"
# units = "imperial"
# Concatinating weather_url_query

query_url=(f"{url}appid={weather_api_key}&units={units}&q=")
query_url

# weather_url_query = (f"{base_url}appid={weather_api_key}&units={units}&q=")
# weather_url_query

# Print statements to begin API retrieval 
#print("Beginning of Data Retrieval")
#print("---------------------------")
#print is working

# Creating empty lists to populate when calling weather_api
city_list = []
cloudiness = []
country = []
date = []
humidity = []
latitude =[]
longitude =[]
max_temp = []
wind_speed = []

#record = 1

# For loop to call the API
for city in cities: 
    try: 
# requesting from API to search each city in the list cities
        response = requests.get(query_url + city).json()
#         response
#         print(json.dumps(response, indent=4,sort_keys= True))
#       Appending information to empty lists as calls are concluded
        city_list.append(response["name"])
        cloudiness.append(response["clouds"]["all"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
        humidity.append(response["main"]["humidity"])
        latitude.append(response["coord"]["lat"])
        longitude.append(response["coord"]["lon"])
        max_temp.append(response["main"]["temp_max"])
        wind_speed.append(response["wind"] ["speed"])
#         #city_record = response["name"]
#         #print(f"Processing {record} | {city_record}")
#         #print(f"{weahter_url_query}&q={city}")
#         #record = record+1
        time.sleep (1.01)
        print(city)
    except KeyError: 
        print("City not found ...")
    continue
        
    

auki
broken hill
atuona
City not found ...
matadi
ushuaia
manbij
bandarbeyla
kemin
thompson
new norfolk
esperance
sechura
hobart
barrow
rikitea
sao joao da barra
plettenberg bay
coahuayana
City not found ...
busselton
caravelas
mahebourg
bambous virieux
albany
mount gambier
upernavik
nikolskoye
fortuna
port alfred
tasiilaq
avarua
vaini
City not found ...
chicama
cape town
tuzha
yellowknife
cannington
jamestown
hermanus
hilo
City not found ...
marawi
arraial do cabo
asau
City not found ...
lazaro cardenas
punta arenas
hithadhoo
longyearbyen
City not found ...
rincon
castro
kaitangata
kingaroy
sao felix do xingu
butaritari
City not found ...
korla
nokaneng
asosa
khatanga
shimoda
puerto ayora
marsh harbour
City not found ...
lagoa
madera
lebu
veglie
leningradskiy
tari
itarema
bluff
marzuq
la ronge
severo-kurilsk
salalah
bon accord
kralendijk
saskylakh
lorengau
City not found ...
kavieng
talnakh
carnarvon
jalu
ketchikan
souillac
cascais
cherskiy
nemuro
City not found ...
City not found ...

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [12]:
weather_dict = {"City": city,
               "Cloudiness": cloudiness,
               "Country": country,
               "Date": date,
               "Humidity": humidity,
               "Latitude": latitude, 
               "Longitude": longitude, 
               "Max Temp": max_temp, 
               "Wind Speed": wind_speed}

weather_df = pd.DataFrame(weather_dict)

weather_df.count()

City          553
Cloudiness    553
Country       553
Date          553
Humidity      553
Latitude      553
Longitude     553
Max Temp      553
Wind Speed    553
dtype: int64

In [5]:
weather_df.to_csv(output_data_file, index=False, header=True)

FileNotFoundError: [Errno 2] No such file or directory: 'output_data/cities.csv'

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression